In [ ]:
# Collaborative Filtering:

In [10]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

In [ ]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [ ]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [ ]:
df['genres'] = df['genres'].str.split('|')
genres_df = df['genres'].apply(pd.Series).stack().reset_index(level=1, drop=True).astype(str)
genres_df = pd.get_dummies(genres_df).groupby(level=0).sum()

In [ ]:
le = LabelEncoder()
df['movieId'] = le.fit_transform(df['movieId'])
df['userId'] = le.fit_transform(df['userId'])

In [ ]:
user_item_matrix = df.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)

In [ ]:
movie_similarity = cosine_similarity(user_item_matrix.T)

In [ ]:
def recommend_movies(user_id, num_recommendations=5):
    rated_movies = user_item_matrix.loc[user_id]
    similar_movies = movie_similarity[rated_movies.index]
    recommendation_scores = similar_movies.dot(rated_movies.values) / similar_movies.sum()
    recommended_movie_indices = recommendation_scores.argsort()[::-1][:num_recommendations]
    recommended_movies = movies_df.iloc[recommended_movie_indices]['title']
    return recommended_movies.tolist()

In [ ]:
def recommend_movies_gui():
    def get_recommendations():
        user_id = int(entry_user_id.get())
        recommended_movies = recommend_movies(user_id)
        recommended_movies_str.set("\n".join(recommended_movies))

    root = tk.Tk()
    root.title("Movie Recommendation System")

    frame = ttk.Frame(root, padding="20")
    frame.grid()

    label_user_id = ttk.Label(frame, text="Enter User ID:")
    label_user_id.grid(row=0, column=0, sticky=tk.W)

    entry_user_id = ttk.Entry(frame, width=10)
    entry_user_id.grid(row=0, column=1)

    button_recommend = ttk.Button(frame, text="Recommend Movies", command=get_recommendations)
    button_recommend.grid(row=1, column=0, columnspan=2, pady=10)

    recommended_movies_str = tk.StringVar()
    label_recommendations = ttk.Label(frame, textvariable=recommended_movies_str, wraplength=300, justify=tk.LEFT)
    label_recommendations.grid(row=2, column=0, columnspan=2)

    root.mainloop()

In [ ]:
#To get Recommendation just enter id of user:
recommend_movies_gui()

In [ ]:
#

In [ ]:
#

In [ ]:
# Content-based :

In [11]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [ ]:
merged_df = pd.merge(ratings_df, movies_df, on='movieId')

In [ ]:
movie_ratings = pd.DataFrame(merged_df.groupby('title')['rating'].mean())

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
movies_df['genres'] = movies_df['genres'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies_df['genres'])

In [ ]:
nn = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='cosine')
nn.fit(tfidf_matrix)

In [ ]:
def get_recommendations(title, nn_model, movies_data, tfidf_model):
    title_tfidf = tfidf_model.transform([title])
    distances, indices = nn_model.kneighbors(title_tfidf)
    recommendations = movies_data.iloc[indices[0], :]['title'].tolist()
    return recommendations

In [ ]:
def recommend_movies():
    movie_title = entry.get()
    if not movie_title:
        messagebox.showwarning("Warning", "Please enter a movie title.")
        return
    
    recommendations = get_recommendations(movie_title, nn, movies_df, tfidf)
    recommendation_text.set(f"Recommendations for '{movie_title}':\n\n" + "\n".join(recommendations))

In [ ]:
root = tk.Tk()
root.title("Movie Recommendation System")

In [ ]:
label = tk.Label(root, text="Enter a Movie Title:")
label.pack(pady=10)

In [ ]:
entry = tk.Entry(root, width=50)
entry.pack()

In [ ]:
button = tk.Button(root, text="Get Recommendations", command=recommend_movies)
button.pack(pady=10)

In [ ]:
recommendation_text = tk.StringVar()
recommendation_label = tk.Label(root, textvariable=recommendation_text, wraplength=400, justify='left')
recommendation_label.pack(pady=10)

In [ ]:
# Run to get Recommendation:
root.mainloop()